In [1]:
import pickle
import numpy as np
from keras.layers import Input, Dense, LSTM, Dropout
from keras_tuner import HyperParameters, RandomSearch
from keras.optimizers import Adam
from keras.models import Model
from pandas import DataFrame
from keras.callbacks import EarlyStopping

2024-03-21 19:55:17.129884: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
with open('../data/processed/bin/sequences/train/X_train.pkl', 'rb') as f:
    X_train: np.ndarray = pickle.load(f)
with open('../data/processed/bin/sequences/train/y_train.pkl', 'rb') as f:
    y_train: np.ndarray = pickle.load(f)

In [3]:
X_train[1].shape

(6, 13)

In [4]:
def model_builder(hp: HyperParameters) -> Model:
    inputs = Input(shape=(X_train[1].shape))

    lstm1 = LSTM(hp.Int('lstm1', 4, 64, 2),return_sequences=True)(inputs)
    
    dropout1 = Dropout(0.2)(lstm1)

    lstm2 = LSTM(hp.Int('lstm2', 4, 64, 2))(lstm1)
    
    # dense = Dense(hp.Int('dense', 8, 256, 4), activation='relu')(lstm2)
   
    output = Dense(units=1)(lstm2)
    model = Model(inputs,output,name='index-price-prediction')
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate',0.001, 0.01, 0.001)),loss='mean_squared_error')
    return model

In [5]:
early_stopping = EarlyStopping(monitor='loss', patience=40, verbose=1, restore_best_weights=True)

In [6]:
tuner = RandomSearch(model_builder, objective='loss', seed = 143, max_trials= 50)

Reloading Tuner from ./untitled_project/tuner0.json


In [7]:
tuner.search(X_train, y_train)

In [8]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [9]:
hypermodel = tuner.hypermodel.build(best_hps)
history = hypermodel.fit(X_train, y_train, epochs=150, callbacks=[early_stopping])

Epoch 1/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0033
Epoch 2/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0024 
Epoch 3/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0024 
Epoch 4/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0027 
Epoch 5/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0024 
Epoch 6/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0024 
Epoch 7/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0021 
Epoch 8/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0025 
Epoch 9/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0022 
Epoch 10/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0024 
Epoch 11/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0024 
Epoch 12/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0022 
Epoch 13/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0022 
Epoch 14/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0022 
Epoch 15/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0025 
Epoch 16/150
3/3 ━━━

In [10]:
DataFrame(history.history).to_csv('../data/hist/training_hist.csv', index_label='epoch')

In [11]:
hypermodel.save('../model/bin/model.keras')